In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Flatten, Dense, GlobalMaxPooling2D
from keras.models import Model
from keras.utils import to_categorical

### Data pre-processing

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()

In [ ]:
print(X_train.shape)
number_of_sample = 20
first_twenty = []
for i in range(number_of_sample):
  first_twenty.append(i)

mask = np.isin(y_train, first_twenty)
y_train_all = y_train[mask]
X_train_all = X_train[mask.squeeze()]


mask = np.isin(y_test, first_twenty)
X_test = X_test[mask.squeeze()]
y_test = y_test[mask]

print(X_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 32, 32, 3)
(2000,)


In [ ]:
X_train, y_train = X_train_all[ : int(len(X_train_all) * 0.9)], y_train_all[ : int(len(X_train_all) * 0.9)]
X_val, y_val = X_train_all[int(len(X_train_all) * 0.9) : ], y_train_all[int(len(X_train_all) * 0.9) : ]

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)

y_train = to_categorical(y_train, 20);
y_test = to_categorical(y_test, 20)
y_val = to_categorical(y_val, 20)

(9000, 32, 32, 3) (9000,)
(2000, 32, 32, 3) (2000,)
(1000, 32, 32, 3) (1000,)


### Model number1 : RestNet50

In [ ]:
from keras.applications import ResNet50

In [ ]:
# def preprocess(images, label):
#   images = tf.image.resize(images, [224, 224])
#   images = images / 255.0
#   return images, label

In [ ]:
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# train_dataset = train_dataset.map(preprocess).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
# val_dataset = val_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
# test_dataset = test_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
resnet = ResNet50(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
resnet.trainable = False
x = resnet.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(resnet.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
print(X_train.shape, y_train.shape)
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

(9000, 32, 32, 3) (9000, 20)
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_5994']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 23s 45ms/step - accuracy: 0.3454 - loss: 0.2221 - val_accuracy: 0.5170 - val_loss: 0.1284
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.5999 - loss: 0.1025 - val_accuracy: 0.5330 - val_loss: 0.1239
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7077 - loss: 0.0794 - val_accuracy: 0.5200 - val_loss: 0.1256
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7654 - loss: 0.0648 - val_accuracy: 0.5480 - val_loss: 0.1274
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8214 - loss: 0.0549 - val_accuracy: 0.5350 - val_loss: 0.1405


In [ ]:
import os

In [ ]:

model.save("resnet50_model.h5")

size_in_bytes = os.path.getsize("resnet50_model.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"ResNet50 model size: {size_in_mb:.2f} MB")

ResNet50 model size: 96.60 MB


In [ ]:
final_epoch = -1  # last epoch

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.5562 - loss: 0.1254
Training Loss: 0.0561
Training Accuracy: 0.8129
Validation Loss: 0.1405
Validation Accuracy: 0.5350
Test Loss: 0.1248
Test Accuracy: 0.5575


###2. VGG16

In [ ]:
from keras.applications import VGG16

In [ ]:
vgg16 = VGG16(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
vgg16.trainable = False
x = vgg16.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(vgg16.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6172']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.2697 - loss: 0.9365 - val_accuracy: 0.4190 - val_loss: 0.2307
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5309 - loss: 0.1630 - val_accuracy: 0.4640 - val_loss: 0.1875
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6586 - loss: 0.1015 - val_accuracy: 0.4730 - val_loss: 0.1848
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7453 - loss: 0.0756 - val_accuracy: 0.4810 - val_loss: 0.1838
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8092 - loss: 0.0596 - val_accuracy: 0.4810 - val_loss: 0.1926


In [ ]:
final_epoch = -1  # last epoch

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4926 - loss: 0.1763
Training Loss: 0.0623
Training Accuracy: 0.7946
Validation Loss: 0.1926
Validation Accuracy: 0.4810
Test Loss: 0.1718
Test Accuracy: 0.5065


In [ ]:

model.save("vgg16.h5")

size_in_bytes = os.path.getsize("vgg16.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"VGG model size: {size_in_mb:.2f} MB")

VGG model size: 57.78 MB


###3. VGG19

In [ ]:
from keras.applications import VGG19

In [ ]:
vgg19 = VGG19(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
vgg19.trainable = False
x = vgg19.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(vgg19.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6194']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.2658 - loss: 0.8857 - val_accuracy: 0.3850 - val_loss: 0.2319
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.4959 - loss: 0.1617 - val_accuracy: 0.4170 - val_loss: 0.1934
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6222 - loss: 0.1095 - val_accuracy: 0.4740 - val_loss: 0.1782
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6918 - loss: 0.0865 - val_accuracy: 0.4770 - val_loss: 0.1750
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7683 - loss: 0.0683 - val_accuracy: 0.4970 - val_loss: 0.1767


In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5038 - loss: 0.1679
Training Loss: 0.0726
Training Accuracy: 0.7492
Validation Loss: 0.1767
Validation Accuracy: 0.4970
Test Loss: 0.1656
Test Accuracy: 0.5085


In [ ]:

model.save("vgg19.h5")

size_in_bytes = os.path.getsize("vgg19.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"VGG model size: {size_in_mb:.2f} MB")

VGG model size: 78.05 MB


### 4. DenseNet

In [ ]:
from keras.applications import DenseNet121

In [ ]:
densenet = DenseNet121(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
densenet.trainable = False
x = densenet.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(densenet.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6219']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 60s 128ms/step - accuracy: 0.1947 - loss: 0.4435 - val_accuracy: 0.3450 - val_loss: 0.1702
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.3966 - loss: 0.1529 - val_accuracy: 0.3920 - val_loss: 0.1514
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.4629 - loss: 0.1359 - val_accuracy: 0.4150 - val_loss: 0.1442
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5002 - loss: 0.1256 - val_accuracy: 0.4400 - val_loss: 0.1406
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.5433 - loss: 0.1163 - val_accuracy: 0.4370 - val_loss: 0.1447


In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.4742 - loss: 0.1332
Training Loss: 0.1185
Training Accuracy: 0.5312
Validation Loss: 0.1447
Validation Accuracy: 0.4370
Test Loss: 0.1352
Test Accuracy: 0.4740


In [ ]:

model.save("densnet121.h5")

size_in_bytes = os.path.getsize("densnet121.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"VGG model size: {size_in_mb:.2f} MB")

VGG model size: 31.17 MB


### 5. MobileNet

In [ ]:
from keras.applications import MobileNet

In [ ]:
mobilenet = MobileNet(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
mobilenet.trainable = False
x = mobilenet.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(mobilenet.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

<ipython-input-150-e1fff43893b0>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet = MobileNet(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6649']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.0605 - loss: 0.3241 - val_accuracy: 0.0940 - val_loss: 0.2011
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.0980 - loss: 0.1998 - val_accuracy: 0.1170 - val_loss: 0.1947
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1189 - loss: 0.1933 - val_accuracy: 0.1240 - val_loss: 0.1913
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1229 - loss: 0.1906 - val_accuracy: 0.1290 - val_loss: 0.1904
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1262 - loss: 0.1901 - val_accuracy: 0.1300 - val_loss: 0.1904


In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.1120 - loss: 0.1919
Training Loss: 0.1901
Training Accuracy: 0.1249
Validation Loss: 0.1904
Validation Accuracy: 0.1300
Test Loss: 0.1911


In [ ]:
model.save("mobilenet.h5")

size_in_bytes = os.path.getsize("mobilenet.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"mobilenet model size: {size_in_mb:.2f} MB")

mobilenet model size: 15.66 MB


### 6. MobileNetV2

In [ ]:
from keras.applications import MobileNetV2

In [ ]:
mobilenetV2 = MobileNetV2(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
mobilenetV2.trainable = False
x = mobilenetV2.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(mobilenetV2.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

<ipython-input-154-127876af6909>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenetV2 = MobileNetV2(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6738']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 19s 39ms/step - accuracy: 0.0577 - loss: 0.3682 - val_accuracy: 0.1060 - val_loss: 0.1981
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.1091 - loss: 0.1956 - val_accuracy: 0.1230 - val_loss: 0.1917
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.1306 - loss: 0.1903 - val_accuracy: 0.1300 - val_loss: 0.1907
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1334 - loss: 0.1885 - val_accuracy: 0.1250 - val_loss: 0.1904
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1400 - loss: 0.1878 - val_accuracy: 0.1310 - val_loss: 0.1902


In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1215 - loss: 0.1897
Training Loss: 0.1877
Training Accuracy: 0.1389
Validation Loss: 0.1902
Validation Accuracy: 0.1310
Test Loss: 0.1901


In [ ]:
model.save("mobilenetv2.h5")

size_in_bytes = os.path.getsize("mobilenetv2.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"mobilenetv2 model size: {size_in_mb:.2f} MB")

mobilenetv2 model size: 12.94 MB


###7. NASNetMobile

In [ ]:
from keras.applications import NASNetMobile

In [ ]:
nasnetmobile = NASNetMobile(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
nasnetmobile.trainable = False
x = nasnetmobile.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(nasnetmobile.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6895']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 68s 129ms/step - accuracy: 0.0966 - loss: 7.5430 - val_accuracy: 0.1350 - val_loss: 1.3513
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.1423 - loss: 1.1001 - val_accuracy: 0.0970 - val_loss: 0.6022
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.1167 - loss: 0.4820 - val_accuracy: 0.0730 - val_loss: 0.4277
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.1269 - loss: 0.3210 - val_accuracy: 0.0630 - val_loss: 0.3282
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.1124 - loss: 0.2592 - val_accuracy: 0.1010 - val_loss: 0.2582


In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.1111 - loss: 0.2484
Training Loss: 0.2506
Training Accuracy: 0.1120
Validation Loss: 0.2582
Validation Accuracy: 0.1010
Test Loss: 0.2496


In [ ]:
model.save("nasnetmobile.h5")

size_in_bytes = os.path.getsize("nasnetmobile.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"nasnetmobile model size: {size_in_mb:.2f} MB")

nasnetmobile model size: 22.87 MB


###8. ResNet101

In [ ]:
from keras.applications import ResNet101

In [ ]:
resnet101 = ResNet101(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
resnet101.trainable = False
x = resnet101.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(resnet101.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_7667']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 43s 78ms/step - accuracy: 0.3362 - loss: 0.2545 - val_accuracy: 0.5000 - val_loss: 0.1283
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6148 - loss: 0.1017 - val_accuracy: 0.5290 - val_loss: 0.1231
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.6939 - loss: 0.0807 - val_accuracy: 0.5320 - val_loss: 0.1243
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7650 - loss: 0.0659 - val_accuracy: 0.5420 - val_loss: 0.1275
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8271 - loss: 0.0534 - val_accuracy: 0.5470 - val_loss: 0.1310


In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - accuracy: 0.5812 - loss: 0.1224
Training Loss: 0.0558
Training Accuracy: 0.8156
Validation Loss: 0.1310
Validation Accuracy: 0.5470
Test Loss: 0.1256


In [ ]:
model.save("resnet101.h5")

size_in_bytes = os.path.getsize("resnet101.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"resnet101 model size: {size_in_mb:.2f} MB")

resnet101 model size: 169.88 MB


###9. ConvNextSamll

In [ ]:
from keras.applications import ConvNeXtSmall

In [ ]:
convnextsmall = ConvNeXtSmall(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
convnextsmall.trainable = False
x = convnextsmall.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(convnextsmall.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
model.save("convnextsmall.h5")

size_in_bytes = os.path.getsize("convnextsmall.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"convnextsmall model size: {size_in_mb:.2f} MB")

### 10. ConvNeXtBase

In [ ]:
from keras.applications import ConvNeXtBase

In [ ]:
convnextbase = ConvNeXtBase(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
convnextbase.trainable = False
x = convnextbase.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'sigmoid', name = 'output-layer')(x)
model = Model(convnextbase.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

In [ ]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
model.save("convnextbase.h5")

size_in_bytes = os.path.getsize("convnextbase.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"convnextbase model size: {size_in_mb:.2f} MB")